In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace
from pyspark.sql.types import FloatType

# Inicjalizacja sesji Spark
spark = SparkSession.builder.master("local[2]").appName("Zadanie_2").getOrCreate()

# Wczytanie pliku zamowienia.txt
file_path = "zamowienia.txt"
df_orders = spark.read.option("header", "true").option("delimiter", ";").csv(file_path)



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/16 22:28:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

In [2]:
df_orders = df_orders.withColumn("Sprzedawca", regexp_replace(col("Sprzedawca"), "ä", "ń"))
df_orders = df_orders.withColumn("Utarg", regexp_replace(col("Utarg"), ",", "."))
df_orders = df_orders.withColumn("Utarg", regexp_replace(col("Utarg"), " z", "").cast(FloatType()))
df_orders.printSchema()
df_orders.show(5)

root
 |-- Kraj: string (nullable = true)
 |-- Sprzedawca: string (nullable = true)
 |-- Data zamowienia: string (nullable = true)
 |-- idZamowienia: string (nullable = true)
 |-- Utarg: float (nullable = true)

+------+----------+---------------+------------+------+
|  Kraj|Sprzedawca|Data zamowienia|idZamowienia| Utarg|
+------+----------+---------------+------------+------+
|Polska|  Kowalski|     16.07.2003|       10248| 440.0|
|Polska|  Sowiński|     10.07.2003|       10249|  NULL|
|Niemcy|   Peacock|     12.07.2003|       10250|  NULL|
|Niemcy| Leverling|     15.07.2003|       10251|654.06|
|Niemcy|   Peacock|     11.07.2003|       10252|  NULL|
+------+----------+---------------+------------+------+
only showing top 5 rows



In [3]:
import shutil
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace
from pyspark.sql.types import FloatType

spark = SparkSession.builder.appName("Data Bucketing and Partitioning").getOrCreate()

df_orders = spark.read.option("header", "true").csv("./df_orders.csv")

# Zadanie 2.1 - Wiaderkowanie danych
bucketed_table_path = "./bucketed_orders"

shutil.rmtree(bucketed_table_path, ignore_errors=True)
spark.sql("DROP TABLE IF EXISTS orders_bucketed")

df_orders.write.bucketBy(16, "idZamowienia").mode("overwrite").saveAsTable("orders_bucketed")

avg_utarg_bucketed = spark.sql("SELECT AVG(Utarg) FROM orders_bucketed").collect()[0][0]
print(f"Średni utarg (bucketed): {avg_utarg_bucketed}")

# Zadanie 2.2 - Partycjonowanie danych
partitioned_table_path = "./partitioned_orders"

shutil.rmtree(partitioned_table_path, ignore_errors=True)
df_orders.write.partitionBy("Kraj").mode("overwrite").csv(partitioned_table_path)

# Zadanie 2.3 - 
start_time = time.time()
avg_utarg_original = spark.sql("SELECT AVG(Utarg) FROM orders_bucketed WHERE Kraj = 'Polska'").collect()[0][0]
original_time = time.time() - start_time
print(f"Średni utarg (oryginalne dane): {avg_utarg_original}")
print(f"Czas wykonania dla danych oryginalnych: {original_time:.2f} sekundy")

partitioned_df = spark.read.option("header", "true").csv(partitioned_table_path)

partitioned_df.printSchema()

partitioned_df = partitioned_df.withColumn(
    "_c5",
    regexp_replace(col("_c5"), ",", ".").cast(FloatType()).alias("Utarg")
)

start_time = time.time()
avg_utarg_partitioned = (
    partitioned_df.filter(col("Kraj") == "Polska")
    .groupBy("Kraj")
    .avg("_c5")
    .collect()[0][1]
)
partitioned_time = time.time() - start_time

print(f"Średni utarg (partycjonowane dane): {avg_utarg_partitioned}")
print(f"Czas wykonania dla danych partycjonowanych: {partitioned_time:.2f} sekundy")


25/01/16 22:28:57 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/opt/spark/work-dir/df_orders.csv.

In [4]:
sample_1 = orders_df.sample(0.1).withColumn("month", col("Data zamowienia").substr(4, 2))
sample_2 = orders_df.sample(0.1).withColumn("netto", round(col("Utarg") / 1.23, 2))
sample_3 = orders_df.sample(0.1).withColumn("Sprzedawca", col("Sprzedawca").upper())
sample_4 = orders_df.sample(0.1).withColumn("waluta", lit("PLN"))

results = {
    "Sredni_Utarg_Wiaderkowane": bucketed_avg,
    "Czas_Wiaderkowanie": bucketed_time,
    "Sredni_Utarg_Original": avg_utarg_original,
    "Czas_Original": original_time,
    "Sredni_Utarg_Partycjonowane": avg_utarg_partitioned,
    "Czas_Partycjonowane": partitioned_query_time,
    "Czas_Partycjonowanie": partition_time
}

results

NameError: name 'orders_df' is not defined